# Generative AI Capstone Project

### Real Estate Assistant - Leveraging AI to Assit in House Hunting

The **purpose** of this project is to create a real estate agent that will be able to assist people in finding their dream homes! Finding a home or a place to rent can be difficult, whether you are a student looking for apartments available and you have to find one before the semester begins or you are switching careers and need to move locations to a new area you know nothing about.

This AI agent will pull data from multiple APIs to help people looking for a new home or apartment. It will show active listings, suggest areas to look into, provide insights into housing market trends, track changes in property values, and even bring in the latest news about the housing market and interest rates that might impact buying decisions. The goal is to make house hunting easier and less stressful by summarizing active listings, offering insights into neighborhoods, and helping you narrow down your options. Once you find homes you're interested in, the agent will provide you with a list of realtors or agencies to contact and set up viewings. It will also give insights into what kind of offers might be appropriate for certain homes and more!

**Significance and Personal Interest** 

Growing up with parents who own a few rental properties, I've always been interested in the time, effort, and thought that goes into the process of searching for, viewing, and eventually purchasing a home. I realized that if there were a way to bridge the gap between the excitement of wanting to start house hunting and the dread of spending hours searching for listings, gaining insights on market trends, and finding a reliable agent to schedule viewings, it would make the entire process much more efficient and would be a valuable product.

Real estate industry trends show that AI adoption is growing rapidly. According to PwC's 2024 Real Estate Report, 60% of real estate firms already use AI in some way, and the AI real estate market is projected to grow to $133 billion by 2030, with a CAGR of 35% (Statista, 2023). One of the more significant statistic found in terms of this use case is that 73% of home buyers begin their search online, relying on AI-driven platforms to help them find the right property (National Association of Realtors, 2023). In addition, Zillow’s AI-based Zestimate is now more accurate than human appraisals 60% of the time (Zillow Research, 2023), this is an extremely significant improvement and gives reason for implementing into this use case!

The AI real estate agent fits within these trends by automating the property search, price analysis, and market insights, reducing the amount of time people have to spend doing manual research. This agent could be implemented by any real estate agency to answer questions about the market and help realtors find listings that match their clients' criteria, saving time and resources


Some **limitations** are as follows:
- The Zillow API key is from Rapid API, this is not affiliated with Zillow, the reason this API was used was because a request was sent in order to get the real Zillow APIs but the request was never approved. 
    - Although, for future implementation or further creation the real APIs could be considered if willing to pay and if coming from a real agency and not just a school project.
- APIs might not have the most up-to-date or comprehensive listings. Free APIs often have limited data compared to premium versions.
- Some APIs may not cover all regions equally, leading to gaps in certain areas. This could be problematic for users searching in less populated or international locations.
- Real estate decisions involve emotions, personal preferences, and qualitative factors that AI cannot always interpret correctly. Users may still need human expertise for nuanced decision-making.
    - This is why the opportunity is not to replace realtors and agencies, merely to assist in the process for all!

 


In [1]:
%pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install jupyter

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -qU "langchain[openai]" langchain

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install langchain openai requests

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install --upgrade --quiet langchain-community

Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.llms import OpenAI
from langchain.agents import AgentExecutor
from dotenv import load_dotenv
from langchain.tools import Tool
from openai import OpenAI
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
import requests
from langchain_community.tools import BraveSearch

import os
load_dotenv()


True

# LLM  

In [7]:
model = init_chat_model("gpt-4o-mini", model_provider="openai")

The model below uses OpenAI to create an LLM that is a helpful realtor assistant, it is told to summarize information pertaining to the questions asked and to be helpful and kind! 

In [8]:
messages = [
    SystemMessage("You are a helpful realtor assistant. You will summarize housing information pertaining to the questions asked. You will ask for clarification if you need it. Analyse trends in the housing market to identify where the best locations are to buy right now. Be kind and helpful!"),
    HumanMessage("What houses are actively for sale in Boston right now?"),
]

model.invoke(messages)

AIMessage(content="I currently don't have real-time access to current listings or databases to provide specific properties. To find actively listed homes for sale in Boston, I recommend checking reliable real estate websites such as Zillow, Realtor.com, or contacting a local real estate agent who can provide you with the most up-to-date information.\n\nIf you need guidance on particular neighborhoods, price ranges, or property types in Boston, feel free to ask!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 73, 'total_tokens': 156, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_709714d124', 'finish_reason': 'stop', 'logprobs': None}, id='run-d5ecd555-1ca0-43f7-8667-ddef408f7d94-0', usage_metadata=

# Tools

The response given above highlghts the need for an agent in this use case. Since LLMs are trained on a static dataset they cannot provide real time information on housing listings, home values to date and recent news to help gain understanding of the market. This is why the following tools will be implemented in order for the agent to pull from the API's and give real time analysis.

## Rent Cast Listing Information

RentCast Listing Information tool will gather housing information for houses actively on the market. Will respond with metrics like days on market, the sale price, address and more! Being able to summarize information of active listings will help people find what they are looking for in a faster manner than scrolling through realtor sites. This will be the beginning of the search to understand what is available.

In [9]:
RENTCAST_API_KEY = os.getenv("RENTCAST_API_KEY")  

def listings_analysis(county: str):
    url = "https://api.rentcast.io/v1/listings/sale"
    headers = {"X-Api-Key": RENTCAST_API_KEY}

    params = {"county" : county}

    #Make the API call
    response = requests.get(url, headers=headers, params=params)

    #Checking if the response was successful
    if response.status_code == 200:
        return response.json()  
    else:
        return {"error": response.json()}  

#Tool for Listings
listings_info_tool_RC = Tool(
    name="RentCast Listings Finder",
    func=listings_analysis,
    description=(
        "Retrieve active listings based on the county provided."
        "You can ask for properties by location (city, state, zip) or address."
    )
)


## RentCast Market Information 

The RentCast Market Tool is a helpful tool that analyzes the housing market trends over time. The marketing data is compiled of aggregate data from the listings data, RentCast removes outlying data in order to proviede realistic overall mean, median and other statistical properties. This data is updated once a day allowing real time analysis of the housing market. This tool allows buyers to understand the trends in the market around them - especially helpful for  use cases of people looking to purchase a place for rent, you want to make sure you find aplace where the ROI will be higher and the market is positively trending in the areas you are looking.

In [10]:
def market_data(zip_code: str):
    url = "https://api.rentcast.io/v1/markets"
    headers = {"X-Api-Key": RENTCAST_API_KEY}

    params = {"zipCode": zip_code}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()  
    else:
        return {"error": response.text}

#Market Data Tool
market_data_tool_RC = Tool(
    name="RentCast Market Data Lookup",
    func=market_data,
    description="Fetches aggregate sale and rental market data (averages, trends) for given location (zip, city, state)."
)

## Zillow Location Information

The Location Information tool allows people who are looking for housing in a new area to get an idea of what neighborhoods are around or what cities they could be looking in. It is very helpful for someone who is just starting to figure out where they want to live - if you have a state in mind you can just ask it to give some suggestions within that state. Combined with the search tool you can gain insights on what the area is like before filtering your housing search for it.

In [11]:
ZILLOW_API_KEY = os.getenv("ZILLOW_API_KEY") 

def get_location_suggestions(query):
    url = "https://zillow-com1.p.rapidapi.com/locationSuggestions"
    headers = {
        "x-rapidapi-key": os.getenv("ZILLOW_API_KEY"),  
        "x-rapidapi-host": "zillow-com1.p.rapidapi.com"
    }
    querystring = {"q": query}
    response = requests.get(url, headers=headers, params=querystring)

    #Checking for errors
    if response.status_code == 200:
        return response.json()  
    else:
        return {"error": response.json()}  

#Tool for Zillow Location Suggestions
zillow_location_tool = Tool(
    name="Zillow Location Suggestions",
    func=get_location_suggestions,
    description="Fetches location suggestions from Zillow based on user input.")

## Zillow Property Information

The Property Information tool allows people who are actively looking for housing to enter an address and get returned a summary of all property details such as square foot, bedrooms, bathrooms etc. This is great for those who find a house for sale that catches their eye and don't want to try to find the listing through multiple sites. This tool will bring up the information needed to get an understanding of the property.

In [12]:
def get_property_details(address: str):
    url = "https://zillow-com1.p.rapidapi.com/property"
    headers = {
        "x-rapidapi-key": ZILLOW_API_KEY,
        "x-rapidapi-host": "zillow-com1.p.rapidapi.com"
    }
    
    querystring = {"address": address}
    
    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        return response.json() 
    else:
        return {"error": response.json()}  

#Tool for Property Details (By address)
property_details_by_address_tool = Tool(
    name="Zillow Property Details Lookup by Address",
    func=get_property_details,
    description="Fetches detailed property information from Zillow by passing the address."
)


## Zillow Zestimate Value Change

This tool is another good one for use cases of people looking to buy a house in a neighborhood for rent for example. You want the home value to be increasing and the neighborhood as a whole the value to increase. By passing the model the url to the property listing it can calculate the percentage change of the homes value through the years and give insights on wether the home is a good buy based on the trends.

In [13]:
def get_zestimate_change(property_url: str):
    url = "https://zillow-com1.p.rapidapi.com/valueHistory/zestimatePercentChange"
    headers = {
        "x-rapidapi-key": ZILLOW_API_KEY,
        "x-rapidapi-host": "zillow-com1.p.rapidapi.com"
    }
    query_params = {"property_url": property_url}
    response = requests.get(url, headers=headers, params=query_params)

    if response.status_code == 200:
        return response.json()  
    else:
        return {"error": response.json(), "status_code": response.status_code}

#Zestimate Percent Change Tool
zestimate_change_tool = Tool(
    name="Zillow Zestimate Percent Change",
    func=get_zestimate_change,
    description="Retrieves historical Zestimate percent change based on a given Property URL."
)



## Zillow Agent Information

Once you find a spot you like, the Zillow Agent Info tool will be able to provide you a list of agents in the location of the homes you would like to see. It provides ratings and reviews in order to understand how previous clients felt and how helpful they will be. The contact information is also given so you can call and book you appointment to see your house!

In [14]:
def find_agent(location :str):
    url = "https://zillow-com1.p.rapidapi.com/findAgentV2"
    headers = {
        "x-rapidapi-key": ZILLOW_API_KEY,
        "x-rapidapi-host": "zillow-com1.p.rapidapi.com"
    }
    
    query_params = {"location" : location}
    
    response = requests.get(url, headers=headers, params=query_params)

    if response.status_code == 200:
        return response.json()  
    else:
        return {"error": response.json()}  

#Zillow Agent Lookup Tool
agent_lookup_tool = Tool(
    name="Zillow Agent Lookup",
    func=find_agent,
    description="Finds real estate agents on Zillow based on their location."
)


## Zillow Property Images 

If you were to find a listing that interests you, with the Zillow property images tool you can now ask for the posted images of the listing, given the zpid. This will output a list of links that will be able to show the listing images. 

In [15]:
def get_property_images(zpid: str):
    url = "https://zillow-com1.p.rapidapi.com/images"
    headers = {
        "x-rapidapi-key": ZILLOW_API_KEY,
        "x-rapidapi-host": "zillow-com1.p.rapidapi.com"
    }
    
    query_params = {"zpid": zpid}
    
    response = requests.get(url, headers=headers, params=query_params)

    if response.status_code == 200:
        return response.json()  
    else:
        return {"error": response.json()}  

#Tool for Zillow Property Images
property_images_tool = Tool(
    name="Zillow Property Images",
    func=get_property_images,
    description="Fetches property images from Zillow using the property ZPID."
)

## Information in the News 

Understanding the housing market includes understanding outside factors that influence it such as interest rates and any housing related news. By using the Information in the News tool, you can gain a deeper understanding of the market to make an informed decsion on whether you would like to buy now or wait. 

In [16]:
NEWS_API_KEY = os.getenv("NEWS_API_KEY")

def fetch_news_about_housing(query: str):
    url = 'https://newsapi.org/v2/everything'
    params = {
        'q': query,  
        'apiKey': NEWS_API_KEY,
        'language': 'en',  #English articles
        'pageSize': 5,     #Limit to 5 articles
    }

    response = requests.get(url, params=params)
    data = response.json()

    articles = data.get('articles', [])
    results = []

    if articles:
        for article in articles:
            results.append({
                "title": article['title'],
                "source": article['source']['name'],
                "url": article['url']
            })
    else:
        results.append("No recent articles found.")

    return results

#News Tool
news_tool = Tool(
    name="News Fetcher",
    func=fetch_news_about_housing,
    description="Fetches the latest news articles related to interest rates or the housing market based on a given query."
)



## Brave Search

Adding the BraveSearch tool was to ensure people could understand the best that they could the locations they are looking at, whether it is a family area, the different neighborhoods and much more.

In [17]:
SEARCH_API = os.getenv("SEARCH_KEY")

#BraveSearch Tool
search_tool = BraveSearch.from_api_key(api_key=SEARCH_API, search_kwargs={"count": 20})

## Model 

In [18]:
# Initialize the LLM given the agent model built in beginning 
llm = model

# Creating the agent using all tools created
property_agent = initialize_agent(
    tools=[search_tool, property_images_tool, news_tool, zestimate_change_tool, zillow_location_tool, market_data_tool_RC, listings_info_tool_RC, agent_lookup_tool, property_details_by_address_tool],
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

/var/folders/qv/51g1ddr90p32d47njb_6jr_80000gn/T/ipykernel_78790/1257992232.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  property_agent = initialize_agent(


# Queries - Messages 

In [19]:
location_listing = property_agent.run("Please give me location sugestions in San Bernadino, California.")
print(location_listing)

/var/folders/qv/51g1ddr90p32d47njb_6jr_80000gn/T/ipykernel_78790/2215646626.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  location_listing = property_agent.run("Please give me location sugestions in San Bernadino, California.")




> Entering new AgentExecutor chain...
I need to find location suggestions in San Bernardino, California that may provide insight into different neighborhoods or areas within that city. 
Action: Zillow Location Suggestions  
Action Input: "San Bernardino, California"  
Observation: {'results': [{'resultType': 'Region', 'display': 'San Bernardino, CA', 'metaData': {'lng': -117.33573536857, 'lat': 34.186483021545, 'city': 'San Bernardino', 'country': 'United States', 'regionType': 'city', 'state': 'CA', 'regionId': 20328, 'county': 'San Bernardino County'}}, {'resultType': 'Region', 'display': 'San Bernardino County, CA', 'metaData': {'lng': -116.17845704124, 'countyFIPS': '06071', 'state': 'CA', 'lat': 34.841438952896, 'regionType': 'county', 'country': 'United States', 'regionId': 3250, 'county': 'San Bernardino County'}}, {'resultType': 'Region', 'display': '92407', 'metaData': {'zipCode': '92407', 'state': 'CA', 'regionId': 96868, 'lat': 34.225113450464, 'city': 'San Bernardino', 'c

In [20]:
brave_search = property_agent.run("Please use brave search to tell me more about the demographics in San Bernardino County, CA.")
print(brave_search)



> Entering new AgentExecutor chain...
I need to find up-to-date information regarding the demographics of San Bernardino County, CA. I'll use the brave search tool to gather relevant demographic data.  
Action: brave_search  
Action Input: "demographics of San Bernardino County, CA"  
Observation: [{"title": "County Profile \u2013 San Bernardino County Community Indicators", "link": "https://indicators.sbcounty.gov/county-profile/", "snippet": "<strong>San</strong> <strong>Bernardino</strong> Regional Parks Department (<strong>County</strong> park acreage); Department of Public Works, Special Districts (Special District park acreage); Southern California Association of Government, custom data request, 2023 (countywide recreational acreage) California Department of Finance, <strong>Demographic</strong> ..."}, {"title": "U.S. Census Bureau QuickFacts: San Bernardino County, California", "link": "https://www.census.gov/quickfacts/fact/table/sanbernardinocountycalifornia/PST045223", "sni

In [21]:
listings_search = property_agent.run("Please give me a list of active listings in San Bernardino, California.")
print(listings_search)



> Entering new AgentExecutor chain...
I need to find active property listings in San Bernardino, California. The appropriate action for this inquiry is to use the RentCast Listings Finder, as it can provide active listings in the specified area.

Action: RentCast Listings Finder  
Action Input: San Bernardino, California  
Observation: [{'id': 'Alden-Rd,-Adelanto,-CA-92301', 'formattedAddress': 'Alden Rd, Adelanto, CA 92301', 'addressLine1': 'Alden Rd', 'addressLine2': None, 'city': 'Adelanto', 'state': 'CA', 'zipCode': '92301', 'county': 'San Bernardino', 'latitude': 34.620779, 'longitude': -117.40612, 'propertyType': 'Land', 'lotSize': 405108, 'status': 'Active', 'price': 175000, 'listingType': 'Standard', 'listedDate': '2024-06-06T00:00:00.000Z', 'removedDate': None, 'createdDate': '2023-01-04T11:08:28.622Z', 'lastSeenDate': '2025-02-23T13:26:00.550Z', 'daysOnMarket': 263, 'mlsName': 'CRMLS', 'mlsNumber': 'SR24113392', 'listingAgent': {'name': 'Clemente De La Torre', 'phone': '818

In [22]:
images_test = property_agent.run("Please give me the links to images for the house with zpid : 17618973")
print(images_test)



> Entering new AgentExecutor chain...
I need to retrieve the property images associated with the given ZPID from Zillow. 

Action: Zillow Property Images  
Action Input: 17618973  
Observation: {'images': ['https://photos.zillowstatic.com/fp/7a537046d0ce06e54a83d041a91341c1-cc_ft_1536.jpg', 'https://photos.zillowstatic.com/fp/81b86ccd52eb9af01b161ac9b3e0292d-cc_ft_1536.jpg', 'https://photos.zillowstatic.com/fp/2944c504d0b87c27ed07a78ebe44e20c-cc_ft_1536.jpg', 'https://photos.zillowstatic.com/fp/23125492de7fd012cd395c4548a3acb6-cc_ft_1536.jpg', 'https://photos.zillowstatic.com/fp/e2a0a8b624cd1d5ed126ec8b4eede56a-cc_ft_1536.jpg', 'https://photos.zillowstatic.com/fp/c7d16e29a670f9c44d1a52e1a0ce4735-cc_ft_1536.jpg', 'https://photos.zillowstatic.com/fp/6679146a2de3603e47749e6b1f640171-cc_ft_1536.jpg', 'https://photos.zillowstatic.com/fp/e5ace59fb5409ee1e464b5a16b2ee3cb-cc_ft_1536.jpg', 'https://photos.zillowstatic.com/fp/93d142da430717d0bd8d23f0315d467b-cc_ft_1536.jpg', 'https://photos.zi

In [23]:

percentage_change_test = property_agent.run("What is the percent change for this property url : https://www.zillow.com/homedetails/40284-Lakeview-Dr-Big-Bear-Lake-CA-92315/17384157_zpid/. Does this percentage chage indicate a healthy housing market or should I wait to buy in this area?")
print(percentage_change_test)



> Entering new AgentExecutor chain...
To determine the percent change for the specified property and assess the housing market conditions in that area, I will retrieve the historical Zestimate percent change for the provided property URL.

Action: Zillow Zestimate Percent Change  
Action Input: https://www.zillow.com/homedetails/40284-Lakeview-Dr-Big-Bear-Lake-CA-92315/17384157_zpid/  
Observation: {'chartData': [{'name': 'This home', 'points': [{'y': -0.0266442745924, 'x': 1456732800000}, {'y': 0.070834934711456, 'x': 1459407600000}, {'y': 0.10952793061733, 'x': 1461999600000}, {'y': 0.069478899240494, 'x': 1464678000000}, {'y': -0.018734818324447, 'x': 1467270000000}, {'y': 0.054148968309164, 'x': 1469948400000}, {'y': -0.024927042424679, 'x': 1472626800000}, {'y': -0.15887585282326, 'x': 1475218800000}, {'y': -0.17336036264896, 'x': 1477897200000}, {'y': -0.13528284430504, 'x': 1480492800000}, {'y': -0.033098414540291, 'x': 1483171200000}, {'y': 0.058856021612883, 'x': 14858496000

In [24]:

zill_property_details2 = property_agent.run("Please use Zillow Property Details to give me property details for 40284 Lakeview Dr, Big Bear Lake, CA. Please also give the market trends for the zip code 92315. ")
print(zill_property_details2)




> Entering new AgentExecutor chain...
I need to gather detailed property information for the specific address provided and also look into the market trends for the zip code 92315. I will first look up the property details on Zillow.

Action: Zillow Property Details Lookup by Address  
Action Input: 40284 Lakeview Dr, Big Bear Lake, CA  
Observation: {'buildingPermits': None, 'contact_recipients': [{'agent_reason': 1, 'zpro': None, 'recent_sales': 1, 'review_count': 42, 'display_name': 'Lisa Bolanos JD and Team Elevate', 'badge_type': 'Premier Agent', 'business_name': 'Elevate Realty', 'rating_average': 5, 'phone': {'prefix': '380', 'areacode': '909', 'number': '9955'}, 'zuid': 'X1-ZUz6wy06zafcw9_1ekgp', 'image_url': 'https://photos.zillowstatic.com/fp/d98e7bdf6c07f9a64c814b4595186383-h_g.jpg'}], 'longitude': -116.922905, 'countyFIPS': '06071', 'imgSrc': 'https://photos.zillowstatic.com/fp/2c4de5269971e0f6d84340efbeb6d2e8-p_d.jpg', 'livingAreaValue': 1132, 'streetAddress': '40284 Lake

In [25]:
agent_response = property_agent.run("Please find me a list of 5 real estate agents in California with high reviews.")
print(agent_response)



> Entering new AgentExecutor chain...
I need to find real estate agents in California who have high reviews. I will search for agents and focus on their ratings and reviews. 

Action: Zillow Agent Lookup  
Action Input: California  
Observation: {'professionals': [{'fullName': 'Tami Pardee', 'profilePhotoSrc': 'https://photos.zillowstatic.com/fp/f9a1e94a0356cb1503809f2985b523e0-h_g.jpg', 'isTopAgent': True, 'salePriceRangeThreeYearMin': 12000, 'isTeamLead': True, 'reviewLink': '/profile/Pardee-Properties#reviews', 'reviewStarsRating': 5, 'reviewExcerptDate': '2025-02-13 10:59:00', 'saleCountLastYear': 242, 'salePriceRangeThreeYearMax': 9500000, 'saleCountAllTime': 3027, 'reviews': '1478 reviews', 'reviewExcerpt': 'Once we closed on the house, Adam continued to be an incredible resource.', 'encodedZuid': 'X1-ZUz0nmomozy2o9_9bpwk', 'phoneNumber': '(310) 861-7719', 'numTotalReviews': 1478, 'profileLink': '/profile/Pardee-Properties', 'location': 'Venice, CA', 'businessName': 'Pardee Pro

In [26]:

market_response = property_agent.run("Please tell me the market trends for the housing market for a location in Boston, MA with zip code 02135. Is this a good market to be buying this house in? Given the trends and comp proces in the area, what would be a good offer price?")
print(market_response)





> Entering new AgentExecutor chain...
I need to gather market data for the housing market in Boston, MA with zip code 02135. This will help understand current trends and assess whether it is a good time to buy. Additionally, I can look into listing prices to gauge what a reasonable offer might be. 

Action: RentCast Market Data Lookup  
Action Input: 02135  
Observation: {'id': '02135', 'zipCode': '02135', 'saleData': {'lastUpdatedDate': '2025-02-23T00:00:00.000Z', 'averagePrice': 937408, 'medianPrice': 788000, 'minPrice': 280000, 'maxPrice': 4500000, 'averagePricePerSquareFoot': 768.86, 'medianPricePerSquareFoot': 730.34, 'minPricePerSquareFoot': 362.41, 'maxPricePerSquareFoot': 1368.42, 'averageSquareFootage': 1329, 'medianSquareFootage': 937, 'minSquareFootage': 318, 'maxSquareFootage': 5790, 'averageDaysOnMarket': 61.07, 'medianDaysOnMarket': 34, 'minDaysOnMarket': 1, 'maxDaysOnMarket': 283, 'newListings': 27, 'totalListings': 118, 'dataByPropertyType': [{'propertyType': 'Condo',

In [27]:
news_response = property_agent.run("Please give me the latest news in interest rates or housing market.")



> Entering new AgentExecutor chain...
To find the latest news about interest rates or the housing market, I will perform a search for recent news articles on these topics. 
Action: News Fetcher  
Action Input: "latest news in interest rates or housing market"  
Observation: [{'title': "Elon Musk's DOGE sends a chill through housing market", 'source': 'Yahoo Entertainment', 'url': 'https://finance.yahoo.com/news/elon-musks-doge-sends-chill-211300446.html'}, {'title': 'Mortgage rates cut below 4% as competition picks up', 'source': 'BBC News', 'url': 'https://www.bbc.com/news/articles/c0e43dqv271o'}]
I have found some recent news articles related to interest rates and the housing market. 

Final Answer: 
1. [Elon Musk's DOGE sends a chill through housing market](https://finance.yahoo.com/news/elon-musks-doge-sends-chill-211300446.html) - Yahoo Entertainment
2. [Mortgage rates cut below 4% as competition picks up](https://www.bbc.com/news/articles/c0e43dqv271o) - BBC News

> Finished ch

### Conclusion 

**Challenges** 

Quite a few challenges in this project came from the API's, the subscriptions and the limitation of the data for some of those. One of the key challenges in this project was the integration of multiple APIs. While the goal is to create a helpful, efficient experience for users by using real-time property listings, market trends, and valuation data, many APIs, only offer limited access through their free plans. These restrictions result in incomplete data, slower updates, and limited geographic coverage, which can impact the accuracy of the information provided to users. Accessing premium versions of these APIs required pricy subscriptions or approval processes. This challenge has meant relying on alternative data sources or working with partial data, potentially limiting the AI agent’s full capabilities.

**Future Enhancements**
This AI real estate agent has the ability to evolve into a comprehensive tool that goes beyond helping users find homes. Future applications could include a negotiations assistant to guide users through making offers and counteroffers, saving them time and effort in the buying process. A predictive feature could forecast housing prices and trends, offering insights into future market conditions and helping users make more informed decisions. A mortgage estimator tool could simplify the financial side by providing users with potential monthly payments based on various factors. Additionally, Gmail integration could enable the agent to directly contact realtors, scheduling viewings or discussing listings in real-time.

In conclusion, implementing this AI agent into real estate firms could save valuable time and resources by assisting in the process for both clients and realtors. By automating the search, filtering, and initial contact steps, realtors can focus more on high-priority tasks, like showings and personalized advice. Clients benefit from a more efficient and less overwhelming house hunting experience, while agencies can optimize their time, ensuring faster responses and better resource allocation.

API Sites 

- https://rapidapi.com/
- https://www.rentcast.io/api?gad_source=1&gclid=Cj0KCQiAq-u9BhCjARIsANLj-s16wG_2-WDav80EtNZCpOkue8r7eBqqKIP31vNPNHIOAbajco56_AQaAplPEALw_wcB
- https://newsapi.org/
- https://brave.com/search/api/

